In [1]:
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
from pygame import mixer

pygame 2.5.2 (SDL 2.28.3, Python 3.11.0)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
model = load_model(r'C:\Users\Pranav\Desktop\jupyter\xception_model.h5')

In [3]:
mixer.init()
sound = mixer.Sound(r'C:\Users\Pranav\Desktop\jupyter\alarm.wav')

In [ ]:


cap = cv2.VideoCapture(0)
Score = 0
# Check if the camera is opened successfully
if not cap.isOpened():
    print("Error: Could not open camera.")
    exit()

# This is preprocessing
while True:
    ret, frame = cap.read()
    height, width = frame.shape[0:2]

    # Check if the frame is empty
    if not ret or frame is None:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=3)
    eyes = eye_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=1)

    cv2.rectangle(frame, (0, height-50), (200, height), (0, 0, 0), thickness=cv2.FILLED)

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, pt1=(x, y), pt2=(x+w, y+h), color=(255, 0, 0), thickness=3)

    for (ex, ey, ew, eh) in eyes:
        cv2.rectangle(frame, pt1=(ex, ey), pt2=(ex+ew, ey+eh), color=(0, 255, 0), thickness=3)

        # Preprocessing
        eye = frame[ey:ey+eh, ex:ex+ew]
        eye = cv2.resize(eye, (80, 80))
        eye = eye / 255
        eye = eye.reshape(1, 80, 80, 3)  # Adjust input shape to match model expectations

        # Preprocessing is done

        prediction = model.predict(eye)
        #closed eyes
        if prediction[0][0] > 0.50:
            cv2.putText(frame, 'closed', (10, height - 20), fontFace=cv2.FONT_HERSHEY_COMPLEX, fontScale=1, color=(255, 255, 255), thickness=1)
            cv2.putText(frame, 'Score' + str(Score), (100, height - 20), fontFace=cv2.FONT_HERSHEY_COMPLEX, fontScale=1, color=(255, 255, 255), thickness=1)
            Score = Score + 1
            if (Score > 5):
                try:
                    sound.play()
                except:
                    pass
        
        #eyes are open
        elif prediction[0][1] > 0.50:
            cv2.putText(frame, 'open', (10, height - 20), fontFace=cv2.FONT_HERSHEY_COMPLEX, fontScale=1, color=(255, 255, 255), thickness=1)
            cv2.putText(frame, 'Score' + str(Score), (100, height - 20), fontFace=cv2.FONT_HERSHEY_COMPLEX, fontScale=1, color=(255, 255, 255), thickness=1)

            Score = Score - 1
            if(Score < 0):
                Score = 0

    cv2.imshow('frame', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):  # Use cv2.waitKey(1) to wait for 1 millisecond
        break

cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 109ms/step
